# TRM
[Link](https://totoro.banrep.gov.co/analytics/saw.dll?Portal&PortalPath=%2Fshared%2FDashboards_T%2FD_Estad%C3%ADsticas%2FEstad%C3%ADsticas&page=Tasas%20de%20cambio%20y%20sector%20externo&NQUser=publico&NQPassword=publico123&lang=es)

Necesita autenticación

### Desde 27/11/1991

In [10]:
from selenium.webdriver.common.by import By
from datetime import datetime
import selenium as sel
import pandas as pd
import time
import glob
import sys
import os

cwd = os.getcwd()
cwd = cwd.replace("Notebooks", "Scripts")
sys.path.insert(0, cwd.replace("\\\\", "\\"))
import base as bs

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

In [30]:
def extraer(start_date: str, end_date: str):
    driver = bs.ejecutar_driver('https://totoro.banrep.gov.co/analytics/saw.dll?Portal&PortalPath=%2Fshared%2FDashboards_T%2FD_Estad%C3%ADsticas%2FEstad%C3%ADsticas&page=Tasas%20de%20cambio%20y%20sector%20externo&NQUser=publico&NQPassword=publico123&lang=es')
    
    frame = driver.find_element(By.XPATH, './/iframe[@id="frame_dashboard"]')
    driver.switch_to.frame(frame)
    driver.find_elements(By.XPATH, '//span[@id="shielded"]//a')[2].click()
    
    time.sleep(40)
    driver.quit()
    
    fuente_archivo = bs.obtener_nombre_descarga('/Users/'+os.getlogin()+'/Downloads')
    
    df = pd.read_excel(fuente_archivo, header=7)
    df.drop(df.tail(4).index, inplace=True)
    df['date'] = df['Fecha (dd/mm/aaaa)'].map(lambda x: x.strftime("%d-%m-%Y"))
    
    try:
        start = df[df['date'] == start_date].index[0]
    except IndexError:
        start = 0
        
    try:
        end = df[df['date'] == end_date].index[0]
    except IndexError:
        end = len(df) - 1
    
    df['Granularidad'] = 'Diario'
    df['Indicador'] = 'TRM'
    df['Unidad'] = 'COP'
    df.rename(columns={'Tasa de cambio representativa del mercado (TRM)': 'Valor'}, inplace=True)

    df = df.loc[start:end, ['date', 'Granularidad', 'Indicador',
                            'Unidad', 'Valor']].reset_index().drop(['index'], axis=1)
    
    os.remove(fuente_archivo)
    
    return df

In [31]:
print(extraer('01-01-1990', '01-01-2040'))

             date Granularidad Indicador Unidad    Valor
0      27-11-1991       Diario       TRM    COP   693.32
1      28-11-1991       Diario       TRM    COP   693.99
2      29-11-1991       Diario       TRM    COP   694.70
3      30-11-1991       Diario       TRM    COP   694.70
4      01-12-1991       Diario       TRM    COP   643.42
...           ...          ...       ...    ...      ...
11503  26-05-2023       Diario       TRM    COP  4470.83
11504  27-05-2023       Diario       TRM    COP  4461.66
11505  28-05-2023       Diario       TRM    COP  4461.66
11506  29-05-2023       Diario       TRM    COP  4461.66
11507  30-05-2023       Diario       TRM    COP  4461.66

[11508 rows x 5 columns]
